# 1.5  Airsim 多无人机控制

AirSim 的多无人机功能支持同时仿真和控制多架无人机，适用于集群任务、协同导航等复杂场景。通过配置文件（如 `settings.json`）可定义无人机的初始位置、类型及通信参数，并利用 Python API 或 ROS 实现分布式控制。例如，用户可通过循环创建多个 `MultirotorClient` 实例，分别连接不同端口，实现多机同步起飞、路径规划及传感器数据共享。  

该功能还支持动态环境交互，如天气模拟、光照变化及自定义地形，可验证无人机在 GPS 拒止环境下的自主导航能力。高级应用中，AirSim 结合 PX4 和 ROS 框架，可构建分布式仿真平台，支持多机协同避障、任务分配等算法测试，为物流配送、搜索救援等场景提供高效验证工具。

1. **协同任务**  
   通过 ROS 或自定义协议实现多机通信，例如任务分配或避障协作。  
2. **分布式仿真**  
   在多台机器上运行多个 AirSim 实例，模拟大规模集群场景。  
3. **可视化调试**  
   使用 RViz 可视化无人机状态和传感器数据。

airsim一般通过配置文件来设置多无人机，当然，也可以通过python代码动态添加，这里以配置文件设置为主：

### **一、配置文件设置**
1. **创建 `settings.json` 文件**  
   在 `~/.airsim/` 目录下新建 `settings.json`，配置 3 个无人机的参数（名称、位置、类型等）：  
   ```json
   {
       "SettingsVersion": 1.2,
       "SimMode": "Multirotor",
       "Vehicles": {
           "UAV1": {
               "VehicleType": "SimpleFlight",
               "X": 0,
               "Y": 0,
               "Z": -2,
               "Yaw": 0
           },
           "UAV2": {
               "VehicleType": "SimpleFlight",
               "X": 5,
               "Y": 0,
               "Z": -2,
               "Yaw": 90
           },
           "UAV3": {
               "VehicleType": "SimpleFlight",
               "X": -5,
               "Y": 0,
               "Z": -2,
               "Yaw": 180
           }
       }
   }
   ```  
   • `X/Y/Z` 定义无人机初始位置（单位：米）  
   • `Yaw` 定义初始朝向（单位：度）

### **二、Python 接口控制**
1. **连接多个无人机**  
   使用不同名称即可
   ```python
    client = airsim.MultirotorClient()
    plane_name = "UAV3" #不同的无人机名称
    client.enableApiControl(True, vehicle_name=plane_name)  #函数调用都类似，添加vehicle_name参数即可
    client.armDisarm(True, vehicle_name=plane_name)
    client.takeoffAsync(vehicle_name=plane_name).join()

·   ```



通过以上步骤，您可以在 AirSim 中快速搭建 3 无人机仿真环境，并通过 Python 实现灵活控制。

将settings.json复制到用户目录下后，可执行后续操作。

## 基本多机控制

In [15]:
import sys
sys.path.append('../external-libraries')
import airsim
import math
import numpy as np
import time

In [16]:
client = airsim.MultirotorClient()  # connect to the AirSim simulator
for i in range(3):
    name = "UAV"+str(i+1)
    client.enableApiControl(True, name)     # 获取控制权
    client.armDisarm(True, name)            # 解锁（螺旋桨开始转动）
    if i != 2:                              # 起飞
        client.takeoffAsync(vehicle_name=name)
    else:
        client.takeoffAsync(vehicle_name=name).join()


for i in range(3):                          # 全部都飞到同一高度层
    name = "UAV" + str(i + 1)
    if i != 2:
        client.moveToZAsync(-3, 1, vehicle_name=name)
    else:
        client.moveToZAsync(-3, 1, vehicle_name=name).join()


In [17]:
origin_x = [0, 2, 4]       # 无人机初始位置
origin_y = [0, 0, 0]

def get_UAV_pos(client, vehicle_name="SimpleFlight"):
    global origin_x
    global origin_y
    state = client.simGetGroundTruthKinematics(vehicle_name=vehicle_name)
    x = state.position.x_val
    y = state.position.y_val
    i = int(vehicle_name[3])
    x += origin_x[i - 1]
    y += origin_y[i - 1]
    pos = np.array([[x], [y]])
    return pos

# 参数设置
v_max = 2     # 无人机最大飞行速度
r_max = 20    # 邻居选择的半径
k_sep = 7     # 控制算法系数
k_coh = 1
k_mig = 1
pos_mig = np.array([[25], [0]])   # 目标位置
v_cmd = np.zeros([2, 9])

for t in range(500):
    for i in range(3):   # 计算每个无人机的速度指令
        name_i = "UAV"+str(i+1)
        pos_i = get_UAV_pos(client, vehicle_name=name_i)
        r_mig = pos_mig - pos_i
        v_mig = k_mig * r_mig / np.linalg.norm(r_mig)
        v_sep = np.zeros([2, 1])
        v_coh = np.zeros([2, 1])
        N_i = 0
        for j in range(3):
            if j != i:
                N_i += 1
                name_j = "UAV"+str(j+1)
                pos_j = get_UAV_pos(client, vehicle_name=name_j)
                if np.linalg.norm(pos_j - pos_i) < r_max:
                    r_ij = pos_j - pos_i
                    v_sep += -k_sep * r_ij / np.linalg.norm(r_ij)
                    v_coh += k_coh * r_ij
                    
        v_sep = v_sep / N_i
        v_coh = v_coh / N_i
        v_cmd[:, i:i+1] = v_sep + v_coh + v_mig

    for i in range(3):   # 每个无人机的速度指令执行
        name_i = "UAV"+str(i+1)
        client.moveByVelocityZAsync(v_cmd[0, i], v_cmd[1, i], -3, 0.1, vehicle_name=name_i)
    time.sleep(0.01)

In [18]:
# 循环结束
client.simPause(False)
for i in range(3):
    name = "UAV"+str(i+1)
    if i != 2:                                              # 降落
        client.landAsync(vehicle_name=name)
    else:
        client.landAsync(vehicle_name=name).join()

for i in range(3):
    name = "UAV" + str(i + 1)
    client.armDisarm(False, vehicle_name=name)              # 上锁
    client.enableApiControl(False, vehicle_name=name)       # 释放控制权

## 多线程运行
前面介绍的是一个进程中，使用join的方式运行，这里采用多线程方式运行，就是一个client负责一个无人机即可。

唯一需要注意的就是，每个线程内部，要自己生成一个client，不能多线程共享一个。

每个无人机的坐标系都是自己的机体坐标系（NED），统一任务时，一般以1个无人机的坐标系为基准，进行坐标转换即可。
<img src="img/s1-5-1.png" width='600px' />



NED（North-East-Down，北东地）坐标系是无人机导航与控制中的核心参考系，其特点与应用如下：

### **1. NED定义与构成**
NED坐标系是一种**局部笛卡尔坐标系**，原点通常设在无人机的起飞点或任务起始位置：

- **X轴（N轴）**：指向地理正北方向；

- **Y轴（E轴）**：指向地理正东方向；

- **Z轴（D轴）**：垂直向下指向地心，遵循右手定则。

与机体坐标系（Body Frame）不同，NED坐标系是**固定在地面或惯性空间的导航坐标系**，用于全局位置和运动状态的描述。而机体坐标系则是固连在无人机上的坐标系，其X轴指向机头，Y轴指向右侧，Z轴垂直向下或向上（根据具体定义）。

---

### **2. 核心功能与应用**
#### **（1）导航与控制**
• **位置与速度基准**：NED坐标系将GPS提供的WGS-84经纬度转换为局部平面坐标，简化路径规划和高度控制。例如，无人机的位置偏移量（如北偏10米、东偏5米）可直接用NED坐标表示。

• **传感器数据融合**：IMU测量的加速度、陀螺仪角速度等机体坐标系数据，需通过坐标转换矩阵映射到NED坐标系，才能用于导航解算。

#### **（2）飞控系统兼容性**
在PX4等飞控系统中，NED坐标系是默认的导航参考系。无人机的初始位置（0,0,0）对应起飞点，所有控制指令（如目标位置、速度）均基于此坐标系生成。

---

### **3. 与其他坐标系的关系**
• **GPS（WGS-84）**：GPS的经纬度需通过地球模型转换到NED坐标系，以适配局部导航需求；

• **ECEF（地心地固系）**：NED是ECEF的局部简化版本，适用于小范围高精度导航，避免地心坐标系的计算复杂度。


In [20]:
#重启airsim环境

import threading

def plane_control(plane_name):
    # connect to the AirSim simulator
    client = airsim.MultirotorClient()
    client.enableApiControl(True, vehicle_name=plane_name)
    client.armDisarm(True, vehicle_name=plane_name)

    client.takeoffAsync(vehicle_name=plane_name).join()
    client.moveToZAsync(-3, 1, vehicle_name=plane_name).join()  # 上升到3米高度，1是速度

    # 初始化4个点的坐标
    points = [airsim.Vector3r(5, 0, -3),
              airsim.Vector3r(5, 5, -3),
              airsim.Vector3r(0, 5, -3),
              airsim.Vector3r(0, 0, -3)]

    client.moveOnPathAsync(points, 1, vehicle_name=plane_name).join()

    client.landAsync(vehicle_name=plane_name).join()
    client.armDisarm(False, vehicle_name=plane_name)
    client.enableApiControl(False, vehicle_name=plane_name)


plane_name_list = []
for i in range(3):
    plane_name_list.append("UAV"+str(i+1))

In [21]:
for plane_name in plane_name_list:
    t = threading.Thread(target=plane_control, args=(plane_name,))
    t.start()

for plane_name in plane_name_list:
    t.join()

参考文档
1. https://microsoft.github.io/AirSim/multi_vehicle/
2. https://zhuanlan.zhihu.com/p/391565827
3. https://blog.csdn.net/kuvinxu/article/details/124467529
4. https://zhuanlan.zhihu.com/p/507304210